# AA Season by Season Analyzer

notebook to analyze AA seasons, and specifically the cali crown rivalry.

In [7]:
import os, sys, subprocess, json, time
from pprint import pprint

import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
xkcd_colors = sns.xkcd_rgb
GOLLY_GRAY = "#272B30"

In [9]:
CUP = 'hellmouth'
ABBR = 'AA'

## Loading Match Data

Start by loading match data from the `data/` dir:

In [10]:
def run_cmd(cmd_str):
    child = subprocess.Popen(cmd_str, shell=True, stdout=subprocess.PIPE)
    output = child.communicate()[0].decode()
    return output

print(run_cmd('ls ../data/'))

clone.sh
gollyx-dragon-data
gollyx-hellmouth-data
gollyx-klein-data
gollyx-pseudo-data
gollyx-rainbow-data
gollyx-star-data
gollyx-toroidal-data



## Method to load regular season data

user specifies the cup and the season.

In [11]:
def fetch_season_data(which_season0):
    seas_file = os.path.join('..', 'data', f'gollyx-{CUP}-data', f'season{which_season0}', 'season.json')
    if not os.path.exists(seas_file):
        raise Exception(f"Error: season {which_season0} not valid: {seas_file} does not exist")
    with open(seas_file, 'r') as f:
        season0_seas = json.load(f)
    return season0_seas

## Method to filter regular season data on a single team

In [12]:
def filter_season_data(seas_data, team_abbr):
    team_abbr = team_abbr.upper()
    team_seas = []
    for day in seas_data:
        for game in day:
            if game['team1Abbr'] == team_abbr or game['team2Abbr'] == team_abbr:
                team_seas.append(game)
                break
    return team_seas

## Test drive

In [13]:
sac = filter_season_data(fetch_season_data(3), ABBR)
print(len(sac))
pprint(sac[0])

49
{'day': 0,
 'gameid': '1b78f20e-1269-4b13-8feb-e4e91787c55c',
 'generations': 1287,
 'isPostseason': False,
 'league': 'Hot League',
 'map': {'cellSize': 7,
         'columns': 120,
         'initialConditions1': '[{"79":[69,70,71]},{"80":[68,71]},{"81":[67]},{"82":[66]}]',
         'initialConditions2': '[{"24":[51]},{"25":[52]},{"26":[53,56]},{"27":[54,55,56]}]',
         'mapName': 'Underground Cave Complex',
         'mapZone1Name': 'Haunted Cave',
         'mapZone2Name': 'Rainbow Cave',
         'mapZone3Name': 'Spider Cave',
         'mapZone4Name': 'Underwater Cave',
         'patternName': 'twomultum',
         'rows': 100,
         'url': '?s1=[{"79":[69,70,71]},{"80":[68,71]},{"81":[67]},{"82":[66]}]&s2=[{"24":[51]},{"25":[52]},{"26":[53,56]},{"27":[54,55,56]}]'},
 'season': 3,
 'team1Abbr': 'SFBS',
 'team1Color': '#e7d7c1',
 'team1Name': 'San Francisco Boat Shoes',
 'team1Score': 96,
 'team1WinLoss': [0, 0],
 'team2Abbr': 'AA',
 'team2Color': '#ff1717',
 'team2Name': 'Al

## Method to assemble schedule tuples - opponent, outcome, score

Extract data from the full game data structure to compile a simpler tuple with opponent, outcome, and score.

In [14]:
def schedule_tuples(team_abbr, season0):
    team_abbr = team_abbr.upper()
    dat = filter_season_data(fetch_season_data(season0), team_abbr)
    schedule_tups = []
    for game in dat:
        if game['team1Abbr'] == team_abbr:
            other_team_name = game['team2Name']
            other_team_abbr = game['team2Abbr']
            
            our_points = game['team1Score']
            other_points = game['team2Score']
            
            our_wl = game['team1WinLoss']
            other_wl = game['team2WinLoss']
            
            if game['team2Score'] > game['team1Score']:
                outcome = 'L'
            else:
                outcome = 'W'
            homeaway = 'H'

        elif game['team2Abbr'] == team_abbr:
            other_team_name = game['team1Name']
            other_team_abbr = game['team1Abbr']
            
            our_points = game['team2Score']
            other_points = game['team1Score']
            
            our_wl = game['team2WinLoss']
            other_wl = game['team1WinLoss']
            
            if game['team1Score'] > game['team2Score']:
                outcome = 'L'
            else:
                outcome = 'W'
            homeaway = 'A'
            
        else:
            raise Exception(f"Could not find team {team_abbr} in season {season0}")

        tup = (other_team_name, other_team_abbr, homeaway, outcome, our_points, other_points, our_wl, other_wl)
        schedule_tups.append(tup)
    return schedule_tups

In [15]:
def print_sched_tuples(team_abbr, season0):
    tups = schedule_tuples(team_abbr, season0)
    print("")
    print(f"Schedule for {team_abbr.upper()}, Season {season0+1}")
    for tup in tups:
        (oppname, oppabbr, homeaway, outcome, ourpts, theirpts, ourwl, theirwl) = tup
        if homeaway=='H':
            matchup = f"{team_abbr:4} ({ourwl[0]:2} - {ourwl[1]:2}) vs {oppabbr:4} ({theirwl[0]:2} - {theirwl[1]:2})"
        elif homeaway=='A':
            matchup = f"{team_abbr:4} ({ourwl[0]:2} - {ourwl[1]:2})  @ {oppabbr:4} ({theirwl[0]:2} - {theirwl[1]:2})"
        print(f"    {matchup}\t{outcome:4}\t{ourpts:4} - {theirpts:4}")
    print("")

In [16]:
print_sched_tuples(ABBR, 3)


Schedule for AA, Season 4
    AA   ( 0 -  0)  @ SFBS ( 0 -  0)	L   	  88 -   96
    AA   ( 0 -  1) vs SFBS ( 1 -  0)	L   	  37 -  172
    AA   ( 0 -  2)  @ MILF ( 0 -  2)	L   	  69 -  263
    AA   ( 0 -  3) vs MILF ( 1 -  2)	L   	 116 -  144
    AA   ( 0 -  4)  @ SFBS ( 2 -  2)	L   	 105 -  197
    AA   ( 0 -  5) vs SFBS ( 3 -  2)	W   	 101 -   63
    AA   ( 1 -  5)  @ DET  ( 4 -  2)	L   	 116 -  125
    AA   ( 1 -  6) vs DET  ( 5 -  2)	L   	  91 -  218
    AA   ( 1 -  7)  @ DET  ( 6 -  2)	L   	 180 -  197
    AA   ( 1 -  8) vs DET  ( 7 -  2)	L   	  34 -  239
    AA   ( 1 -  9)  @ DET  ( 8 -  2)	W   	 162 -  103
    AA   ( 2 -  9) vs DET  ( 8 -  3)	W   	 137 -  101
    AA   ( 3 -  9)  @ SFBS ( 4 -  8)	L   	  18 -  302
    AA   ( 3 - 10) vs SFBS ( 5 -  8)	W   	 241 -   78
    AA   ( 4 - 10)  @ DECO ( 5 -  9)	W   	 274 -   95
    AA   ( 5 - 10) vs DECO ( 5 - 10)	W   	 107 -   91
    AA   ( 6 - 10)  @ SFBS ( 7 -  9)	L   	  12 -  264
    AA   ( 6 - 11) vs SFBS ( 8 -  9)	L   	  91 -  139
 

In [17]:
print_sched_tuples(ABBR, 13)


Schedule for AA, Season 14
    AA   ( 0 -  0)  @ DECO ( 0 -  0)	W   	  82 -   29
    AA   ( 1 -  0) vs DECO ( 0 -  1)	L   	 111 -  175
    AA   ( 1 -  1)  @ MILF ( 0 -  2)	L   	 100 -  105
    AA   ( 1 -  2) vs MILF ( 1 -  2)	W   	 226 -  129
    AA   ( 2 -  2)  @ MILF ( 1 -  3)	W   	 282 -   43
    AA   ( 3 -  2) vs MILF ( 1 -  4)	W   	 215 -   54
    AA   ( 4 -  2)  @ DET  ( 4 -  2)	L   	  36 -  184
    AA   ( 4 -  3) vs DET  ( 5 -  2)	L   	  46 -  214
    AA   ( 4 -  4)  @ DET  ( 6 -  2)	W   	 126 -   82
    AA   ( 5 -  4) vs DET  ( 6 -  3)	W   	  79 -   75
    AA   ( 6 -  4)  @ DET  ( 6 -  4)	L   	  67 -  283
    AA   ( 6 -  5) vs DET  ( 7 -  4)	L   	  70 -  226
    AA   ( 6 -  6)  @ DECO ( 5 -  7)	L   	 126 -  200
    AA   ( 6 -  7) vs DECO ( 6 -  7)	W   	 217 -   71
    AA   ( 7 -  7)  @ DET  ( 9 -  5)	W   	 219 -  152
    AA   ( 8 -  7) vs DET  ( 9 -  6)	W   	 199 -  131
    AA   ( 9 -  7)  @ DECO ( 7 -  9)	W   	 111 -   72
    AA   (10 -  7) vs DECO ( 7 - 10)	W   	 100 -   48


In [18]:
print_sched_tuples(ABBR, 20)


Schedule for AA, Season 21
    AA   ( 0 -  0)  @ ORL  ( 0 -  0)	L   	  75 -  140
    AA   ( 0 -  1) vs ORL  ( 1 -  0)	L   	  36 -  317
    AA   ( 0 -  2)  @ ABQ  ( 0 -  2)	L   	  83 -  151
    AA   ( 0 -  3) vs ABQ  ( 1 -  2)	W   	 100 -   56
    AA   ( 1 -  3) vs ORL  ( 3 -  1)	L   	 129 -  219
    AA   ( 1 -  4)  @ ORL  ( 4 -  1)	W   	 251 -  130
    AA   ( 2 -  4)  @ BB   ( 0 -  6)	L   	 127 -  237
    AA   ( 2 -  5) vs BB   ( 1 -  6)	W   	 202 -  133
    AA   ( 3 -  5)  @ SGE  ( 5 -  3)	L   	  54 -  138
    AA   ( 3 -  6) vs SGE  ( 6 -  3)	L   	  37 -  257
    AA   ( 3 -  7) vs BB   ( 1 -  9)	L   	  93 -  218
    AA   ( 3 -  8)  @ BB   ( 2 -  9)	W   	 140 -   80
    AA   ( 4 -  8) vs BB   ( 2 - 10)	L   	  73 -  233
    AA   ( 4 -  9)  @ BB   ( 3 - 10)	L   	  89 -  129
    AA   ( 4 - 10)  @ DET  ( 8 -  6)	W   	 154 -   54
    AA   ( 5 - 10) vs DET  ( 8 -  7)	W   	 211 -  116
    AA   ( 6 - 10)  @ MILF ( 9 -  7)	L   	  58 -  338
    AA   ( 6 - 11) vs MILF (10 -  7)	L   	  44 -  171


## Method to assemble matchup tuples - specific opponent, outcome, score

In [19]:
def matchup_tuples(team_abbr, opp_abbr, season0):
    team_abbr, opp_abbr = team_abbr.upper(), opp_abbr.upper()
    dat = filter_season_data(fetch_season_data(season0), team_abbr)
    matchup_tups = []
    for game in dat:
        skip = False
        if game['team1Abbr'] == team_abbr and game['team2Abbr'] == opp_abbr:
            other_team_name = game['team2Name']
            other_team_abbr = game['team2Abbr']
            
            our_points = game['team1Score']
            other_points = game['team2Score']
            
            our_wl = game['team1WinLoss']
            other_wl = game['team2WinLoss']
            
            if game['team2Score'] > game['team1Score']:
                outcome = 'L'
            else:
                outcome = 'W'
            homeaway = 'H'

        elif game['team2Abbr'] == team_abbr and game['team1Abbr'] == opp_abbr:
            other_team_name = game['team1Name']
            other_team_abbr = game['team1Abbr']
            
            our_points = game['team2Score']
            other_points = game['team1Score']
            
            our_wl = game['team2WinLoss']
            other_wl = game['team1WinLoss']
            
            if game['team1Score'] > game['team2Score']:
                outcome = 'L'
            else:
                outcome = 'W'
            homeaway = 'A'
            
        else:
            skip = True
            
        if not skip:
            tup = (other_team_name, other_team_abbr, homeaway, outcome, our_points, other_points, our_wl, other_wl)
            matchup_tups.append(tup)
    return matchup_tups

In [20]:
def print_matchup_tuples(team_abbr, opp_abbr, season0):
    tups = matchup_tuples(team_abbr, opp_abbr, season0)
    print("")
    print(f"Schedule for {team_abbr.upper()} vs. {opp_abbr.upper()}, Season {season0+1}")
    wins = sum([1 for tup in tups if tup[3]=='W'])
    losses = sum([1 for tup in tups if tup[3]=='L'])
    print(f"W-L record for {team_abbr}: {wins}-{losses}")
    for tup in tups:
        (oppname, oppabbr, homeaway, outcome, ourpts, theirpts, ourwl, theirwl) = tup
        if homeaway=='H':
            matchup = f"{team_abbr:4} ({ourwl[0]:2} - {ourwl[1]:2}) vs {oppabbr:4} ({theirwl[0]:2} - {theirwl[1]:2})"
        elif homeaway=='A':
            matchup = f"{team_abbr:4} ({ourwl[0]:2} - {ourwl[1]:2})  @ {oppabbr:4} ({theirwl[0]:2} - {theirwl[1]:2})"
        print(f"    {matchup}\t{outcome:4}\t{ourpts:4} - {theirpts:4}")
    print("")

## Delawarehouse Fire Crown AA-DECO

In [26]:
print_matchup_tuples(ABBR, 'DECO', 3)


Schedule for AA vs. DECO, Season 4
W-L record for AA: 5-4
    AA   ( 4 - 10)  @ DECO ( 5 -  9)	W   	 274 -   95
    AA   ( 5 - 10) vs DECO ( 5 - 10)	W   	 107 -   91
    AA   ( 6 - 12)  @ DECO ( 6 - 12)	W   	  85 -    6
    AA   ( 7 - 12) vs DECO ( 6 - 13)	L   	 153 -  208
    AA   ( 7 - 13)  @ DECO ( 7 - 13)	L   	  44 -   88
    AA   ( 7 - 14) vs DECO ( 8 - 13)	L   	 153 -  210
    AA   ( 8 - 16)  @ DECO (10 - 14)	W   	 225 -   97
    AA   ( 9 - 16) vs DECO (10 - 15)	L   	  87 -   97
    AA   (14 - 34) vs DECO (18 - 30)	W   	 185 -   63



In [28]:
for season0 in range(0, 3):
    print_matchup_tuples(ABBR, 'DECO', season0)


Schedule for AA vs. DECO, Season 1
W-L record for AA: 3-6
    AA   ( 1 -  1) vs DECO ( 1 -  1)	W   	 235 -    4
    AA   ( 2 -  1)  @ DECO ( 1 -  2)	L   	  22 -  256
    AA   ( 2 -  2)  @ DECO ( 2 -  2)	L   	  68 -  113
    AA   ( 2 -  3) vs DECO ( 3 -  2)	W   	 259 -   36
    AA   ( 4 -  8) vs DECO ( 7 -  5)	L   	  53 -  100
    AA   ( 4 -  9)  @ DECO ( 8 -  5)	L   	  34 -   72
    AA   ( 8 - 14)  @ DECO (13 -  9)	L   	   2 -  331
    AA   ( 8 - 15) vs DECO (14 -  9)	W   	 122 -   32
    AA   (22 - 26)  @ DECO (24 - 24)	L   	  12 -  406


Schedule for AA vs. DECO, Season 2
W-L record for AA: 3-5
    AA   ( 7 -  3)  @ DECO ( 5 -  5)	L   	   9 -  404
    AA   ( 7 -  4) vs DECO ( 6 -  5)	W   	 429 -    0
    AA   (12 -  6) vs DECO ( 9 -  9)	W   	 352 -   10
    AA   (13 -  6)  @ DECO ( 9 - 10)	L   	   7 -  264
    AA   (13 -  7)  @ DECO (10 - 10)	L   	   7 -  237
    AA   (13 -  8) vs DECO (11 - 10)	W   	 314 -   30
    AA   (14 -  8) vs DECO (11 - 11)	L   	  87 -   94
    AA   (14 -  9

In [29]:
for season0 in range(3, 24):
    print_matchup_tuples(ABBR, 'DECO', season0)


Schedule for AA vs. DECO, Season 4
W-L record for AA: 5-4
    AA   ( 4 - 10)  @ DECO ( 5 -  9)	W   	 274 -   95
    AA   ( 5 - 10) vs DECO ( 5 - 10)	W   	 107 -   91
    AA   ( 6 - 12)  @ DECO ( 6 - 12)	W   	  85 -    6
    AA   ( 7 - 12) vs DECO ( 6 - 13)	L   	 153 -  208
    AA   ( 7 - 13)  @ DECO ( 7 - 13)	L   	  44 -   88
    AA   ( 7 - 14) vs DECO ( 8 - 13)	L   	 153 -  210
    AA   ( 8 - 16)  @ DECO (10 - 14)	W   	 225 -   97
    AA   ( 9 - 16) vs DECO (10 - 15)	L   	  87 -   97
    AA   (14 - 34) vs DECO (18 - 30)	W   	 185 -   63


Schedule for AA vs. DECO, Season 5
W-L record for AA: 4-6
    AA   ( 3 -  1)  @ DECO ( 1 -  3)	L   	  66 -  125
    AA   ( 3 -  2) vs DECO ( 2 -  3)	L   	  93 -  110
    AA   ( 5 -  7)  @ DECO ( 6 -  6)	L   	  85 -   98
    AA   ( 5 -  8) vs DECO ( 7 -  6)	L   	 122 -  194
    AA   ( 8 - 14)  @ DECO (11 - 11)	W   	 265 -   84
    AA   ( 9 - 14) vs DECO (11 - 12)	W   	 157 -   63
    AA   (10 - 14)  @ DECO (11 - 13)	L   	  67 -  190
    AA   (10 - 15

## Alewife-Sacramento



In [30]:
for season0 in range(0, 24):
    print_matchup_tuples(ABBR, 'SAC', season0)


Schedule for AA vs. SAC, Season 1
W-L record for AA: 1-3
    AA   (13 - 19) vs SAC  (17 - 15)	L   	 128 -  144
    AA   (13 - 20)  @ SAC  (18 - 15)	L   	 104 -  118
    AA   (21 - 25)  @ SAC  (23 - 23)	L   	 143 -  159
    AA   (21 - 26) vs SAC  (24 - 23)	W   	 130 -  102


Schedule for AA vs. SAC, Season 2
W-L record for AA: 5-3
    AA   (19 - 15) vs SAC  (16 - 18)	W   	 204 -  144
    AA   (20 - 15)  @ SAC  (16 - 19)	L   	 107 -  145
    AA   (20 - 16)  @ SAC  (17 - 19)	L   	  34 -  438
    AA   (20 - 17) vs SAC  (18 - 19)	W   	 381 -   28
    AA   (25 - 19)  @ SAC  (21 - 23)	W   	 170 -   54
    AA   (26 - 19) vs SAC  (21 - 24)	W   	 163 -   95
    AA   (27 - 19)  @ SAC  (21 - 25)	W   	 177 -   70
    AA   (28 - 19) vs SAC  (21 - 26)	L   	  81 -  112


Schedule for AA vs. SAC, Season 3
W-L record for AA: 3-3
    AA   (12 - 16)  @ SAC  (15 - 13)	L   	  64 -  111
    AA   (12 - 17) vs SAC  (16 - 13)	W   	 472 -    0
    AA   (16 - 22)  @ SAC  (18 - 20)	L   	  37 -  230
    AA   (16 -

## Alewife-Detroit AA-DET

grape chews - arsonists

In [32]:
for season0 in range(0, 24):
    print_matchup_tuples(ABBR, 'DET', season0)


Schedule for AA vs. DET, Season 1
W-L record for AA: 3-3
    AA   ( 5 - 11) vs DET  (10 -  6)	W   	 335 -    7
    AA   ( 6 - 11)  @ DET  (10 -  7)	L   	   9 -  301
    AA   ( 6 - 12)  @ DET  (11 -  7)	L   	  64 -  137
    AA   ( 6 - 13) vs DET  (12 -  7)	W   	 207 -   41
    AA   ( 9 - 15)  @ DET  (13 - 11)	L   	  85 -  142
    AA   ( 9 - 16) vs DET  (14 - 11)	W   	 176 -   23


Schedule for AA vs. DET, Season 2
W-L record for AA: 5-2
    AA   ( 0 -  0)  @ DET  ( 0 -  0)	W   	 190 -  123
    AA   ( 1 -  0) vs DET  ( 0 -  1)	W   	 104 -   60
    AA   ( 6 -  2)  @ DET  ( 3 -  5)	L   	   4 -   77
    AA   ( 6 -  3) vs DET  ( 4 -  5)	W   	 132 -   10
    AA   (14 - 10) vs DET  (11 - 13)	W   	 131 -   16
    AA   (15 - 10)  @ DET  (11 - 14)	W   	  33 -   23
    AA   (28 - 20)  @ DET  (25 - 23)	L   	  55 -   78


Schedule for AA vs. DET, Season 3
W-L record for AA: 2-4
    AA   ( 5 -  7)  @ DET  ( 6 -  6)	L   	  19 -  322
    AA   ( 5 -  8) vs DET  ( 7 -  6)	W   	 123 -   88
    AA   ( 8 -


Schedule for AA vs. DET, Season 23
W-L record for AA: 3-1
    AA   (15 - 13)  @ DET  (14 - 14)	W   	 137 -   39
    AA   (16 - 13) vs DET  (14 - 15)	W   	 188 -   80
    AA   (17 - 13)  @ DET  (14 - 16)	W   	 162 -  109
    AA   (18 - 13) vs DET  (14 - 17)	L   	  82 -  152


Schedule for AA vs. DET, Season 24
W-L record for AA: 5-7
    AA   ( 0 -  0)  @ DET  ( 0 -  0)	L   	   4 -  409
    AA   ( 0 -  1) vs DET  ( 1 -  0)	L   	   6 -  235
    AA   ( 2 -  2)  @ DET  ( 3 -  1)	W   	 174 -  103
    AA   ( 3 -  2) vs DET  ( 3 -  2)	L   	 128 -  133
    AA   (10 - 10)  @ DET  (13 -  7)	W   	 200 -   74
    AA   (11 - 10) vs DET  (13 -  8)	W   	 166 -  116
    AA   (12 - 12)  @ DET  (15 -  9)	W   	 267 -   53
    AA   (13 - 12) vs DET  (15 - 10)	L   	  49 -  269
    AA   (13 - 15)  @ DET  (16 - 12)	L   	  96 -  120
    AA   (13 - 16) vs DET  (17 - 12)	L   	 120 -  157
    AA   (20 - 26)  @ DET  (26 - 20)	W   	 319 -   56
    AA   (21 - 26) vs DET  (26 - 21)	L   	  66 -  176



## Alewife-Milwaukee AA-MILF



In [33]:
for season0 in range(0, 24):
    print_matchup_tuples(ABBR, 'MILF', season0)


Schedule for AA vs. MILF, Season 1
W-L record for AA: 2-4
    AA   ( 3 -  3) vs MILF ( 2 -  4)	L   	  13 -  171
    AA   ( 3 -  4)  @ MILF ( 3 -  4)	L   	  36 -  224
    AA   ( 3 -  5)  @ MILF ( 4 -  4)	L   	 130 -  180
    AA   ( 3 -  6) vs MILF ( 5 -  4)	W   	 112 -  106
    AA   ( 4 - 10) vs MILF ( 8 -  6)	W   	 158 -   10
    AA   ( 5 - 10)  @ MILF ( 8 -  7)	L   	   7 -   43


Schedule for AA vs. MILF, Season 2
W-L record for AA: 3-3
    AA   ( 2 -  0) vs MILF ( 1 -  1)	L   	  77 -  247
    AA   ( 2 -  1)  @ MILF ( 2 -  1)	W   	 104 -   95
    AA   ( 8 -  4) vs MILF ( 5 -  7)	W   	 201 -   21
    AA   ( 9 -  4)  @ MILF ( 5 -  8)	L   	  14 -  172
    AA   (16 - 10) vs MILF (13 - 13)	W   	 378 -    0
    AA   (17 - 10)  @ MILF (13 - 14)	L   	  34 -  334


Schedule for AA vs. MILF, Season 3
W-L record for AA: 2-4
    AA   ( 0 -  2) vs MILF ( 1 -  1)	W   	 271 -   15
    AA   ( 1 -  2)  @ MILF ( 1 -  2)	L   	   4 -  338
    AA   (11 - 13)  @ MILF (13 - 11)	L   	  11 -  350
    AA   (1


Schedule for AA vs. MILF, Season 23
W-L record for AA: 3-1
    AA   ( 0 -  2)  @ MILF ( 1 -  1)	W   	  90 -   53
    AA   ( 1 -  2) vs MILF ( 1 -  2)	W   	 227 -  167
    AA   (19 - 15)  @ MILF (15 - 19)	L   	  64 -  131
    AA   (19 - 16) vs MILF (16 - 19)	W   	 188 -  150


Schedule for AA vs. MILF, Season 24
W-L record for AA: 2-3
    AA   (17 - 19)  @ MILF (18 - 18)	L   	  91 -  189
    AA   (17 - 20) vs MILF (19 - 18)	W   	 131 -   55
    AA   (18 - 22)  @ MILF (20 - 20)	W   	  80 -   75
    AA   (19 - 22) vs MILF (20 - 21)	L   	  21 -  163
    AA   (21 - 27) vs MILF (25 - 23)	L   	  92 -  150



## Alewife-Orlando AA-ORL


In [34]:
for season0 in range(0, 24):
    print_matchup_tuples(ABBR, 'ORL', season0)


Schedule for AA vs. ORL, Season 1
W-L record for AA: 3-3
    AA   (12 - 16)  @ ORL  (12 - 16)	L   	  30 -   78
    AA   (12 - 17) vs ORL  (13 - 16)	L   	  78 -   85
    AA   (12 - 18) vs ORL  (14 - 16)	W   	 132 -  110
    AA   (13 - 18)  @ ORL  (14 - 17)	L   	  17 -  138
    AA   (16 - 22) vs ORL  (19 - 19)	W   	 152 -   58
    AA   (17 - 22)  @ ORL  (19 - 20)	W   	 138 -   51


Schedule for AA vs. ORL, Season 2
W-L record for AA: 2-2
    AA   (18 - 12) vs ORL  (18 - 12)	W   	 274 -   24
    AA   (19 - 12)  @ ORL  (18 - 13)	L   	  21 -  319
    AA   (21 - 17)  @ ORL  (22 - 16)	L   	  61 -  355
    AA   (21 - 18) vs ORL  (23 - 16)	W   	 382 -   52


Schedule for AA vs. ORL, Season 3
W-L record for AA: 3-3
    AA   (13 - 17)  @ ORL  (16 - 14)	L   	  33 -  120
    AA   (13 - 18) vs ORL  (17 - 14)	W   	 309 -    6
    AA   (16 - 20)  @ ORL  (20 - 16)	L   	  29 -  190
    AA   (16 - 21) vs ORL  (21 - 16)	L   	  86 -  129
    AA   (19 - 25)  @ ORL  (24 - 20)	W   	  81 -   60
    AA   (20 -